## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the "pax" column, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at [neobase in Github](https://github.com/alexprengere/neobase))

Bonus point: Solve this problem using pandas (instead of any other approach)


Suggestion: follow the below plan of action:

* Get familiar with the data
* Select columns of interest
* Decide what to do with NaNs

* Make processing plan
* Develop code that works with a sample

* Adjust the code to work with Big data
* Test big data approach on a sample

* Run program with big data


## 1) Get familiar with data

In [ ]:
#nos interesan las columnas airport y pax. Al final hay que hacer la suma de pax, teniendo en cuenta las negativas.
#pero primero exploramos un poco los datos, viendo como están las columnas,  viendo vacías, etc.
#para familiarizarse con los datos, lo primero sacamos una muestra.

In [1]:
import pandas as pd

bookings_file='bookings.csv.bz2'
searches_file='searches.csv.bz2'

sample=pd.read_csv(bookings_file, nrows=100000,sep='^')
#tenemos que ver el delimitador que usa el archivo.
#no pasa nada por que esté comprimido, pandas lo detecta y aplica el descompresor necesario. No funciona con Zip pq en zip.

In [2]:
#para que salgan todas las columnas: pd.options.display (ahí hay muchas opciones para jugar que vemos)
pd.options.display.max_columns=None
sample.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [3]:
sample.shape
# 100000 filas pq le hemos dicho nosotros que coja este numero de filas.

(100000, 38)

In [4]:
sample.describe()

,duration,distance,intl,pax,year,month
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.0,100000.000000
mean,22960.511580,476.904440,0.662550,0.509630,2013.0,2.191040
std,46979.388683,1780.890791,0.472842,2.219013,0.0,1.260222
min,51.000000,0.000000,0.000000,-80.000000,2013.0,1.000000
25%,3370.750000,0.000000,0.000000,-1.000000,2013.0,1.000000
50%,8594.000000,0.000000,1.000000,1.000000,2013.0,3.000000
75%,21535.500000,0.000000,1.000000,1.000000,2013.0,3.000000
max,791131.000000,19012.000000,1.000000,70.000000,2013.0,5.000000


In [5]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 38 columns):
act_date               100000 non-null object
source                 100000 non-null object
pos_ctry               100000 non-null object
pos_iata               100000 non-null object
pos_oid                100000 non-null object
rloc                   100000 non-null object
cre_date               100000 non-null object
duration               100000 non-null int64
distance               100000 non-null int64
dep_port               100000 non-null object
dep_city               100000 non-null object
dep_ctry               100000 non-null object
arr_port               100000 non-null object
arr_city               100000 non-null object
arr_ctry               100000 non-null object
lst_port               100000 non-null object
lst_city               100000 non-null object
lst_ctry               100000 non-null object
brd_port               100000 non-null object
brd_city          

In [6]:
sample.columns
#es una caos, hay espacios en blanco. ¿como los quitamos?
#column es como index, tiene metodos propios, como str. Podemos explorar estos metodos sample.columns.str.
#nos podria valer el replace o mejor el strip

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [7]:
#este método elimina espacios en blanco de los extremos (ambos)
sample.columns.str.strip()

Index(['act_date', 'source', 'pos_ctry', 'pos_iata', 'pos_oid', 'rloc',
       'cre_date', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry',
       'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry',
       'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry',
       'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route', 'carrier',
       'bkg_class', 'cab_class', 'brd_time', 'off_time', 'pax', 'year',
       'month', 'oid'],
      dtype='object')

### What if we dont want to read the whole file?

Options:

* prepare the sample

* read_csv with nrows option

Clean the column names

## 2) Select the columns of interest 

In [8]:
sample=sample[['arr_port','pax','year']]
#nos quedamos tambien con el año por si no hay bookings en 2013

## 3) What to do with NaN?



In the sample everything might be ok, but we should prepare for NaN case

In [9]:
#si nos encontramos nan, en este caso lo mejor es quitarlos
#si fuera un modelo de ML tendríamos que rellenarlos o imputarlos pq le harían falta pero en este caso
#estamos haciendo reportings, cuentas. SI no se a que aeropuerto llega una persona no la cuento y listo, así que se descartan
sample.count()
# no hay nan, mejor

arr_port    100000
pax         100000
year        100000
dtype: int64

## 4) Make processing plan
1) get only the bookings from 2013

2) group by arr_port, sum

3) sort 

4) get top 10

#### 4.1) Get only the booking from 2013

In [10]:
sample2013= sample[sample['year']==2013]
sample2013.shape

(100000, 3)

#### 4.2) group by arr_port, sum

In [12]:
#sample2013.groupby('arr_port').sum() Así nos saldrá el resto de columnas, mejor especificamos la columna
paxes_per_airport=sample2013.groupby('arr_port')['pax'].sum()
#de esta forma se agrupa y luego se suma lo agrupado
#sample2013.groupby('arr_port').sum()['pax'] con esto se calcula la suma de todas las columnas y luego se extrae la columna. Es mucho mas eficiente el primer caso

In [13]:
#paxes_per_airport es una serie, para ordenarla podemos usar directamente sort_values sin decirle el by
paxes_per_airport.sort_values(ascending=False).head(10)
#ESTE SERÍA EL TOP 10

arr_port
LHR         1006
MCO          861
JFK          795
LAX          761
BKK          747
LAS          732
SFO          705
ORD          686
CDG          676
DXB          587
Name: pax, dtype: int64

In [ ]:
#TENEMOS EL PROTOTIPO, PARA PODER ADAPTAR ESTO AL ARCHIVO COMPLETO TENEMOS QUE USAR LOS CHUNCKS

## 5) Adjust the code to work with Big data


Hint: check out https://pandas.pydata.org/pandas-docs/stable/io.html#io-chunking

In [14]:
#Vamos a volver a usar una muestra y vamos a hacer los chunks, cuando tengamos el proceso definido lo corremos con todo
bookings_file='bookings.csv.bz2'
sample=pd.read_csv(bookings_file,nrows=100000,sep='^',chunksize=10000)
#El nº de chunks va a ser el entero nrows/chunksize.

In [15]:
for chunk in sample:
    print(type(chunk))
    #cada chunk es un df como podemos ver

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [17]:
# Para adaptar el código, sólamente tenemos que buscar una forma de agregar los resultados parciales.
# Cada chunk, trabajamos con el y trabajamos de una forma mucho mas comprimida. Ejemlpo: pasa de ser el nº de linas de cada chunk a el nº de aeropuertos

partial_results=[]

for chunk in sample:
    #nos quedamos para empezar con las 3 columnas
    chunk=chunk[['arr_port','pax','year']].dropna()#hacemos dropna por si acaso
    chunk=chunk[sample['year']==2013]
    actual_result=chunk.groupby('arr_port')['pax'].sum()
    #aqui no nos vamos a quedar con el top 10 ordenado pq no actuamos sobre lo general, si no sobre cada chunk
    partial_results.append(actual_result)

partial_results
#En el primer bucle ya he recorrido el archivo por eso sale vacío, se queda en la ultima posicion, para arreglarlo 
#volvemos a leer el erchivo

[]

In [19]:
sample=pd.read_csv(bookings_file,nrows=100000,sep='^',chunksize=10000)
partial_results=[]

for chunk in sample:
    #nos quedamos para empezar con las 3 columnas
    chunk=chunk[['arr_port','pax','year']].dropna()#hacemos dropna por si acaso
    chunk=chunk[chunk['year']==2013]
    actual_result=chunk.groupby('arr_port')['pax'].sum()
    #aqui no nos vamos a quedar con el top 10 ordenado pq no actuamos sobre lo general, si no sobre cada chunk
    partial_results.append(actual_result)

partial_results

[arr_port
 AAL          0
 ABQ         12
 ABV          2
 ABZ         -6
 ACC          4
 ADD          7
 ADZ          2
 AEP          7
 AER          1
 AES          3
 AFA          2
 AGA          0
 AGP          8
 AGU          4
 AGX          2
 AJA          7
 AJF          2
 AKL          2
 ALA          6
 ALB          2
 ALC          9
 ALG         26
 AMD          2
 AMM         16
 AMS         36
 ANC          4
 ANF          2
 ANU          0
 AOE          1
 AOI          0
             ..
 XNB          2
 XQP          2
 YAY         -2
 YCD          3
 YEG         15
 YHD         -1
 YHZ          4
 YMM          0
 YNB          0
 YOW          0
 YQM          0
 YQQ          2
 YQT          1
 YQU          1
 YTM          4
 YUL         14
 YVO          1
 YVR         17
 YWG          6
 YXE          1
 YXS          2
 YXY          1
 YYC         15
 YYJ          3
 YYZ         34
 YZF          4
 ZAG         -4
 ZNZ          1
 ZRH         16
 ZYL         -1
 Name: pax, Le

### We have to read the whole file, but with nrows we are reading always the first N rows


In [26]:
%%time
#Como se puede hacer esto:
#con bucle de joins (merge)
#con concat

#Cuando tenemos que juntar varios DF es mas eficiente pd.concat(partial_results)
#para agregarlos por aeropuerto con .groupby('arr_port').sum()
pd.concat(partial_results).groupby('arr_port').sum().sort_values(ascending=False).head(10)

#AHORA SÍ ESTAMOS LISTO PARA CORRERLO SOBRE TODO EL DATASET

CPU times: user 7.39 ms, sys: 256 µs, total: 7.65 ms
Wall time: 14.4 ms


arr_port
LHR         1006
MCO          861
JFK          795
LAX          761
BKK          747
LAS          732
SFO          705
ORD          686
CDG          676
DXB          587
Name: pax, dtype: int64

#### Now we need to put together the results from all the chunks

Options:

* df.append()

* pd.concat()
    

#### 1) df.append()

#### 2) pd.concat()

## Complete Solution

#quitamos el nrows y le metemos un chuksize mas grande, intentando que cada chunk size sea lo más pequeño posible
data=pd.read_csv(bookings_file,sep='^',chunksize=1e6)
partial_results=[]

for chunk in data:
    #nos quedamos para empezar con las 3 columnas
    chunk=chunk[['arr_port','pax','year']].dropna()#hacemos dropna por si acaso
    chunk=chunk[chunk['year']==2013]
    actual_result=chunk.groupby('arr_port')['pax'].sum()
    #aqui no nos vamos a quedar con el top 10 ordenado pq no actuamos sobre lo general, si no sobre cada chunk
    partial_results.append(actual_result)

all_result=pd.concat(partial_results)
total_arrivals=all_result.groupby('arr_port').sum()
top10=total_arrivals.sort_values(ascending=False).head(10)
top10

In [27]:
%%time
#ahora vamos a ver los registros de tiempo y ver por donde va
from datetime import datetime

data=pd.read_csv(bookings_file,sep='^',chunksize=1e6)

partial_results=[]

for chunk_num,chunk in enumerate(data):
    print('Start processing chunk %d at %s' % (chunk_num, datetime.now()))
    #nos quedamos para empezar con las 3 columnas
    chunk=chunk[['arr_port','pax','year']].dropna()#hacemos dropna por si acaso
    chunk=chunk[chunk['year']==2013]
    actual_result=chunk.groupby('arr_port')['pax'].sum()
    #aqui no nos vamos a quedar con el top 10 ordenado pq no actuamos sobre lo general, si no sobre cada chunk
    partial_results.append(actual_result)

all_result=pd.concat(partial_results)
total_arrivals=all_result.groupby('arr_port').sum()
top10=total_arrivals.sort_values(ascending=False).head(10)
top10

Start processing chunk 0 at 2018-06-01 19:05:59.302966
Start processing chunk 1 at 2018-06-01 19:06:24.060963
Start processing chunk 2 at 2018-06-01 19:06:49.038545
Start processing chunk 3 at 2018-06-01 19:07:14.010931
Start processing chunk 4 at 2018-06-01 19:07:39.097063


<string>:2: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.


Start processing chunk 5 at 2018-06-01 19:08:04.581477
Start processing chunk 6 at 2018-06-01 19:08:29.629677
Start processing chunk 7 at 2018-06-01 19:08:54.590076
Start processing chunk 8 at 2018-06-01 19:09:21.663239
Start processing chunk 9 at 2018-06-01 19:09:48.735861
Start processing chunk 10 at 2018-06-01 19:09:49.045883
CPU times: user 4min 1s, sys: 10.6 s, total: 4min 11s
Wall time: 4min 16s


In [28]:
top10
#Este es el resultado final, el primero es LHR

arr_port
LHR         88809.0
MCO         70930.0
LAX         70530.0
LAS         69630.0
JFK         66270.0
CDG         64490.0
BKK         59460.0
MIA         58150.0
SFO         58000.0
DXB         55590.0
Name: pax, dtype: float64

In [ ]:
# CHUNKS ES LA SOLUCION GENERAL, PERO HAY UN ARGUMENTO DE LA FUNCION READ_CSV (usecols). 
#Con esto solo cargamos en memoria solo las columnas que nos interesan. Pasamos de 32 a 3 columnas.
# tardará menos.
#No hay que hacer la agregacion ni todos los pasos intermedios
#df=pd.read_csv(bookings_file,sep='^',usecols=['pax','arr_port','year'])
#res=df.groupby('arr_port').sum().sort_values(ascending=False).head(10)
#res

In [ ]:
#import sys
#sys.getsizeof(df)
#nos dice el espacio que ocupa en disco el argumento

In [34]:
#BONUS POINT
# en terminal: pip install neobase. Neobase es una BBDD de todos los aeropuertos del mundo, para ver donde estan
# cuando acabe de instalar
import neobase

In [38]:
nb=neobase.NeoBase()
nb.get('JFK')['name']

'John F. Kennedy International Airport'

In [41]:
#Con funcion lambda:
#top10.index.map(lambda airport: nb.get(airport.strip())['name'])
#con una funcion quedará mas explicito:
def get_airport_name_from_IATA(IATA_code):
    airport_data=nb.get(IATA_code.strip())
    return airport_data['name']

top10.index.map(get_airport_name_from_IATA)

Index(['London Heathrow Airport', 'Orlando International Airport',
       'Los Angeles International Airport', 'McCarran International Airport',
       'John F. Kennedy International Airport',
       'Paris Charles de Gaulle Airport', 'Suvarnabhumi Airport',
       'Miami International Airport', 'San Francisco International Airport',
       'Dubai International Airport'],
      dtype='object', name='arr_port')